# COMPUTER-VISION-CHESS

- Eryk Ptaszyński 151950
- Eryk Walter 151931

In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random

In [2]:
class Board:
    def __init__(self, x, y, width, height, mask, contours, color = (255, 0, 0)):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.mask = mask
        self.contours = contours
        self.color = color

    def draw(self, frame):
        frame = cv2.addWeighted(frame, 1, self.mask, 0.2, 0)
        cv2.putText(frame, "Board", (self.x, self.y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.drawContours(frame, self.contours, -1, (0, 255, 0), 4)

        return frame

In [3]:
class Piece:
    def __init__(self, x, y, radius, color = (255, 0, 0)):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color

In [4]:
class Pieces:
    def __init__(self, white_pieces: list, black_pieces: list,buffer_size: int) -> None:
        self.white_pieces = white_pieces
        self.black_pieces = black_pieces
        self.white_circles = []
        self.black_circles = []
        self.buffer_size = buffer_size
    
    def add_pieces(self, white_pieces: list, black_pieces: str):
        self.white_pieces.append(white_pieces)
        self.black_pieces.append(black_pieces)

        if len(self.white_pieces) > self.buffer_size:
            self.white_pieces.pop(0)
        if len(self.black_pieces) > self.buffer_size:
            self.black_pieces.pop(0)
    
    def white_pieces_mask(self, frame):
        masks = []
        for pieces in self.white_pieces:
            mask = np.zeros(frame.shape[:2], dtype=np.uint8)
            for piece in pieces:
                cv2.circle(mask, (piece.x, piece.y), piece.radius, 255, -1)
            masks.append(mask)
        mask = np.median(masks, axis=0).astype(np.uint8)
        return mask

    def black_pieces_mask(self, frame):
        masks = []
        for pieces in self.black_pieces:
            mask = np.zeros(frame.shape[:2], dtype=np.uint8)
            for piece in pieces:
                mask = cv2.circle(mask, (piece.x, piece.y), piece.radius, 255, -1)
            masks.append(mask)
        mask = np.median(masks, axis=0).astype(np.uint8)
        return mask
    
    def draw(self, frame):
        circles = cv2.HoughCircles(self.white_pieces_mask(frame), cv2.HOUGH_GRADIENT, 1, 20, param1=10, param2=10, minRadius=5, maxRadius=40)
        if circles is not None:
            self.white_circles = circles[0].astype(np.uint32)
        else:
            self.white_circles = None
        
        circles = cv2.HoughCircles(self.black_pieces_mask(frame), cv2.HOUGH_GRADIENT, 1, 20, param1=10, param2=10, minRadius=5, maxRadius=40)
        if circles is not None:
            self.black_circles = circles[0].astype(np.uint32)
        else:
            self.black_circles = None

        if self.white_circles is not None:
            for circle in self.white_circles:
                frame = cv2.circle(frame, (circle[0], circle[1]), circle[2], (255, 0, 0), 2)
        if self.black_circles is not None:
            for circle in self.black_circles:
                frame = cv2.circle(frame, (circle[0], circle[1]), circle[2], (0, 0, 255), 2)
        return frame

In [5]:
class Clock:
    def __init__(self, x, y, width, height, mask, contours, color = (255, 0, 0)):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.mask = mask
        self.contours = contours
        self.color = color
    
    def draw(self, frame):
        frame = cv2.addWeighted(frame, 1, self.mask, 0.2, 0)
        cv2.putText(frame, "Clock", (self.x, self.y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.drawContours(frame, self.contours, -1, (0, 255, 255), 4)
        return frame

In [6]:
class BoardDetectors:
    def __init__(self):
        self.strategies = {
            'canny_board_detection': self.canny_board_detection
        }

    def canny_board_detection(self, frame) -> Board:
        edges = cv2.Canny(frame, 100, 200)
        dilated = cv2.morphologyEx(edges, cv2.MORPH_DILATE, np.ones((3, 3)))

        connected_components = cv2.connectedComponentsWithStats(dilated)
        largest_component = np.argmax(connected_components[2][1:, cv2.CC_STAT_AREA]) + 1

        largest_component_mask = connected_components[1] == largest_component
        largest_component_mask = cv2.morphologyEx(largest_component_mask.astype(np.uint8), cv2.MORPH_CLOSE, np.ones((101, 101)))
        largest_component_mask = cv2.morphologyEx(largest_component_mask.astype(np.uint8), cv2.MORPH_OPEN, np.ones((101, 101)), iterations=2)
        largest_component_mask = largest_component_mask.astype(np.uint8) * 255
        largest_component_mask = largest_component_mask[..., np.newaxis]

        contours, _ = cv2.findContours(largest_component_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) == 0:
            return None
        x, y, w, h = cv2.boundingRect(contours[0])

        mask_color = cv2.cvtColor(largest_component_mask, cv2.COLOR_GRAY2BGR)
        colored_mask = np.where(mask_color == 255, np.array([0, 255, 0]), 0)
        colored_mask = colored_mask.astype('uint8')
        board = Board(x, y, w, h, colored_mask, contours)
        return board
    
    def __call__(self, frame, method):
        return self.strategies[method](frame)

In [7]:
class PieceDetectors:
    def __init__(self):
        self.strategies = {
            'all_pieces': self.all_pieces,
        }
    
    def all_pieces(self, frame, board: Board, pieces: Pieces = None) -> Pieces:
        white_pieces = []
        black_pieces = []
        detected_pieces = []
        if board is not None:
            frame = frame[board.y:board.y+board.height, board.x:board.x+board.width]
        
        frame = cv2.GaussianBlur(frame, (3, 3), 0)

        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 40, param1=100, param2=40, minRadius=10, maxRadius=30)
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for circle in circles[0, :]:
                detected_pieces.append(Piece(circle[0], circle[1], circle[2]))

        for piece in detected_pieces:
            piece.color = self.get_piece_color(frame, piece)
            if board is not None:
                piece.x += board.x
                piece.y += board.y
            if piece.color == (255, 255, 255):
                white_pieces.append(piece)
            else:
                black_pieces.append(piece)
        
        pieces.add_pieces(white_pieces, black_pieces)
        return pieces
    
    def get_piece_color(self, frame, piece):
        piece_mask = np.zeros_like(frame)
        cv2.circle(piece_mask, (piece.x, piece.y), piece.radius, (255, 255, 255), -1)
        piece_mask = cv2.cvtColor(piece_mask, cv2.COLOR_RGB2GRAY)
        piece_mask = np.where(piece_mask == 255, 1, 0)
        piece_mask = piece_mask[..., np.newaxis]
        piece_mask = piece_mask.astype('uint8')

        piece_color = cv2.mean(frame, mask=piece_mask)
        if np.mean(piece_color) > 48:
            return (255, 255, 255)
        else:
            return (0, 0, 0)
    
    def __call__(self, frame, board, pieces, method) -> Pieces:
        return self.strategies[method](frame, board, pieces)

In [8]:
class ClockDetectors:
    def __init__(self):
        self.strategies = {
            'detect_clock': self.canny_clock_detection,
        }

    def canny_clock_detection(self, frame, board) -> Clock:
        edges = cv2.Canny(frame, 100, 200)
        dilated = cv2.morphologyEx(edges, cv2.MORPH_DILATE, np.ones((3, 3)))

        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if x > board.x and y < board.y and 1.1 < w / h < 2.5 and w > 100 and h > 100:
                mask = np.zeros_like(frame)
                cv2.drawContours(mask, [contour], -1, (0, 255, 255), -1)
                
                return Clock(x, y, w, h, mask, contour)
        return None
    
    def __call__(self, frame, board, method) -> Clock:
        return self.strategies[method](frame, board)

In [9]:
class Evaluate:
    def __init__(self):
        self.strategies = {
            'count_pieces': self.count_pieces,
        }
    
    def count_pieces(self, pieces: Pieces):
        if pieces.white_circles is None or pieces.black_circles is None:
            return 0
        return len(pieces.white_circles) - len(pieces.black_circles)
    
    def draw(self, frame, board, evaluation):
        if board is None:
            return frame
        cv2.putText(frame, "Evaluation: " + str(evaluation), (board.x + 300, board.y + board.height + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        return frame
    
    def __call__(self, pieces: Pieces, method):
        return self.strategies[method](pieces)

In [10]:
class MoveDetector:
    def __init__(self):
        self.strategies = {
            'detect_move': self.detect_move,
        }
    
    def detect_move(self, board: Board):
        approximation = cv2.approxPolyDP(board.contours[0], 0.01 * cv2.arcLength(board.contours[0], True), True)
        if len(approximation) != 4:
            return True
        bounding_rect = cv2.boundingRect(approximation)
        aspect_ratio = bounding_rect[2] / bounding_rect[3]
        if not 0.9 < aspect_ratio < 1.1:
            return True
        return False
    
    def draw(self, frame):
        frame = cv2.putText(frame, "Piece move", (120, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        return frame
    
    def __call__(self, board: Board, method):
        return self.strategies[method](board)

In [11]:
class ClockTapDetector:
    def __init__(self):
        self.strategies = {
            'detect_tap': self.detect_tap,
        }
    
    def detect_tap(self, clock: Clock):
        if clock is None:
            return True
        approximation = cv2.approxPolyDP(clock.contours[0], 0.01 * cv2.arcLength(clock.contours[0], True), True)
        if len(approximation) != 4:
            return False
        bounding_rect = cv2.boundingRect(approximation)
        aspect_ratio = bounding_rect[2] / bounding_rect[3]
        if not 15/9 < aspect_ratio < 17/7:
            return False
        return True
    
    def draw(self, frame):
        frame = cv2.putText(frame, "Clock tap", (120, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        return frame
    
    def __call__(self, clock: Clock, method):
        return self.strategies[method](clock)

In [12]:
class Video:
    def __init__(self, video_path) -> None:
        self.video = cv2.VideoCapture(video_path)
        self.video.set(cv2.CAP_PROP_CONVERT_RGB, 1)
        self.fps = int(self.video.get(cv2.CAP_PROP_FPS))
        self.frame_count = int(self.video.get(cv2.CAP_PROP_FRAME_COUNT))
        self.width = int(self.video.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.video.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [13]:
class MediaManager:
    def __init__(self):
        self.strategies = {
            "load_image": self.load_image,
            "show_image": self.show_image,
        }
    
    def load_image(self, path: str):
        return cv2.imread(path, cv2.COLOR_BGR2RGB)
    
    def show_image(self, frames: list):
        plt.figure(figsize=(20, 20))
        for i, frame in enumerate(frames, start=1):
            plt.subplot(1, len(frames), i)
            plt.imshow(frame, cmap='gray')
            plt.axis('off')
        plt.show()
        return None
    
    def get_random_frames_from_video(self, video, n_frames=4):
        frames = []
        frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        for i in range(n_frames):
            video.set(cv2.CAP_PROP_POS_FRAMES, random.randint(0, frame_count))
            ret, frame = video.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
        video.set(cv2.CAP_PROP_POS_FRAMES, 0)
        return frames

    def rerecord_video_to_n_parts(self, input_path: str, output_paths: list, n_parts=3):
        assert len(output_paths) == n_parts

        input_video = Video(input_path)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        output_videos = [cv2.VideoWriter(output_path, fourcc, input_video.fps, (input_video.width, input_video.height)) for output_path in output_paths]
        part_length = input_video.frame_count // n_parts
        for i in range(n_parts):
            for j in range(part_length):
                ret, frame = input_video.video.read()
                output_videos[i].write(frame)

        for output_video in output_videos:
            output_video.release()
        input_video.video.release()

        return None
    
    def record_n_random_video_segments(self, input_path: str, output_path: str, n_segments=4, segment_length=60):
        board_detector = BoardDetectors()
        piece_detector = PieceDetectors()
        clock_detector = ClockDetectors()
        
        move_detector = MoveDetector()
        clock_tap_detector = ClockTapDetector()
        evaluator = Evaluate()

        input_video = Video(input_path)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        output_video = cv2.VideoWriter(output_path, fourcc, input_video.fps, (input_video.width, input_video.height))
        pieces = Pieces([], [], 5)
        
        for i in range(n_segments):
            input_video.video.set(cv2.CAP_PROP_POS_FRAMES, random.randint(0, input_video.frame_count - segment_length))
            for j in range(segment_length):
                ret, frame = input_video.video.read()
                board = board_detector(frame, 'canny_board_detection')
                pieces = piece_detector(frame, board, pieces, 'all_pieces')
                clock = clock_detector(frame, board, 'detect_clock')
                evaluation = evaluator(pieces, 'count_pieces')

                if board is not None:
                    frame = board.draw(frame)

                if pieces is not None:
                    pieces.draw(frame)
                
                if clock is not None:
                    frame = clock.draw(frame)
                
                if move_detector(board, 'detect_move'):
                    move_detector.draw(frame)
                
                if clock_tap_detector(clock, 'detect_tap'):
                    clock_tap_detector.draw(frame)
                
                frame = evaluator.draw(frame, board, evaluation)

                output_video.write(frame)

        output_video.release()
        input_video.video.release()
    
    def record_full_video(self, input_path: str, output_path: str):
        board_detector = BoardDetectors()
        piece_detector = PieceDetectors()
        clock_detector = ClockDetectors()

        move_detector = MoveDetector()
        clock_tap_detector = ClockTapDetector()
        evaluator = Evaluate()

        input_video = Video(input_path)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        output_video = cv2.VideoWriter(output_path, fourcc, input_video.fps, (input_video.width, input_video.height))
        pieces = Pieces([], [], 5)

        while True:
            ret, frame = input_video.video.read()
            if not ret:
                break
            board = board_detector(frame, 'canny_board_detection')
            pieces = piece_detector(frame, board, pieces, 'all_pieces')
            clock = clock_detector(frame, 'detect_clock')
            evaluation = evaluator(pieces, 'count_pieces')

            if board is not None:
                frame = board.draw(frame)

            if pieces is not None:
                pieces.draw(frame)
            
            if clock is not None:
                frame = clock.draw(frame)
            
            if move_detector(board, 'detect_move'):
                move_detector.draw(frame)
            
            if clock_tap_detector(clock, 'detect_tap'):
                clock_tap_detector.draw(frame)

            frame = evaluator.draw(frame, board, evaluation)
            output_video.write(frame)

        output_video.release()
        input_video.video.release()

In [15]:
media_manager = MediaManager()
media_manager.record_n_random_video_segments('input/EASY.mp4', 'output.avi', 4, 120)

In [1]:
media_manager = MediaManager()
for name in ['easy_1', 'easy_2', 'easy_3']:
    media_manager.record_full_video(f'input/{name}.avi', f'output/{name}.avi')

for name in ['medium_1', 'medium_2', 'medium_3']:
    media_manager.record_full_video(f'input/{name}.avi', f'output/{name}.avi')

for name in ['hard_1', 'hard_2', 'hard_3']:
    media_manager.record_full_video(f'input/{name}.avi', f'output/{name}.avi')

NameError: name 'MediaManager' is not defined